In [1]:
import torch
import torchvision
from torch import nn
import torchvision.transforms as transforms
from VGG_Face_torch import VGG_Face_torch
import argparse
import torch.optim as optim
from torch.autograd import Variable

In [2]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
batch_size = 16
test_batch_size =16
epochs = 100
lr = 0.001
momentum = 0.9
seed=1
log_interval = 30
kwargs = {'num_workers': 4, 'pin_memory': True}


In [3]:
transform_train = transforms.Compose([transforms.RandomHorizontalFlip(),
                                 transforms.Resize(256),
                                #  transforms.RandomResizedCrop((224,224), scale=(0.875, 1.125), ratio=(1.0, 1.0)),
                                #  transforms.CenterCrop(224),
                                 transforms.RandomCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.507395516207, ),(0.255128989415, ))
                                ])

transform_test  = transforms.Compose([transforms.Resize(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.507395516207, ),(0.255128989415, ))
                                ])


train_data = torchvision.datasets.ImageFolder('./train',transform=transform_train)
test_data = torchvision.datasets.ImageFolder('./test',transform=transform_test)
                                            
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data,   batch_size=test_batch_size, shuffle=False, **kwargs)

In [4]:
class average_meter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [5]:
class VGG_Net(nn.Module):
    def __init__(self, model):
        super(VGG_Net, self).__init__()

        self.pre_model = nn.Sequential(*list(model.children())[:-1])
        # self.dropout = nn.Dropout(p=0.8)
        self.classifier = nn.Linear(4096, 7)

    def forward(self, x):
        x = self.pre_model(x)
        # x = self.dropout(x)
        x = self.classifier(x)

        return x 

In [6]:
model_emotion = VGG_Face_torch
model_emotion.load_state_dict(torch.load('VGG_Face_torch.pth'))
model = VGG_Net(model_emotion).cuda()


loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum ,weight_decay= 0.0005,nesterov=True)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=20, gamma=0.1)

In [7]:
def train(epoch):
    
    losses = average_meter()
    accuracy = average_meter()
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).cuda(), Variable(target).cuda()
        output = model(data)
        loss = loss_function(output, target)
        losses.update(loss.data, data.size(0))

        pred = output.data.max(1)[1]
        prec = pred.eq(target.data).cpu().sum()
        accuracy.update(float(prec) / data.size(0), data.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {}\t'
                  'Batch: [{:5d}/{:5d} ({:3.0f}%)]\t'                     
                  'Loss: {:.6f}'.format(
                      epoch, batch_idx * len(data), len(train_data),
                      100. * batch_idx / len(train_loader), losses.val))
            print('Training accuracy:', accuracy.val )

In [8]:
def test():
    losses = average_meter()
    accuracy = average_meter()

    model.eval()

    for data, target in test_loader:

        data, target = Variable(data,volatile=True).cuda(), Variable(target,volatile=True).cuda()
        output = model(data)

        loss = loss_function(output, target)
        losses.update(loss.data, data.size(0))

        pred = output.data.max(1)[1]
        prec = pred.eq(target.data).cpu().sum()
        accuracy.update(float(prec) / data.size(0), data.size(0))

    print('\nTest: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        losses.avg, int(accuracy.sum), len(test_data), 100. * accuracy.avg))

    return accuracy.avg

In [ ]:
def main():

    best_model = model
    best_accuray = 0.0

    for epoch in range(1, epochs + 1):

        train(epoch)
        val_accuracy = test()
        scheduler.step()


        if best_accuray < val_accuracy:
            best_model   = model
            best_accuray = val_accuracy


    print ("The best model has an accuracy of " + str(best_accuray))
    torch.save(best_model.state_dict(), 'best.model')


if __name__ == '__main__':
    main()

Train Epoch: 1	Batch: [    0/ 4954 (  0%)]	Loss: 1.940064
Training accuracy: 0.3125
Train Epoch: 1	Batch: [  480/ 4954 ( 10%)]	Loss: 0.651697
Training accuracy: 0.875
Train Epoch: 1	Batch: [  960/ 4954 ( 19%)]	Loss: 0.284743
Training accuracy: 0.9375
Train Epoch: 1	Batch: [ 1440/ 4954 ( 29%)]	Loss: 0.222524
Training accuracy: 0.9375
Train Epoch: 1	Batch: [ 1920/ 4954 ( 39%)]	Loss: 0.344153
Training accuracy: 0.9375
Train Epoch: 1	Batch: [ 2400/ 4954 ( 48%)]	Loss: 0.368033
Training accuracy: 0.875
Train Epoch: 1	Batch: [ 2880/ 4954 ( 58%)]	Loss: 0.656362
Training accuracy: 0.6875
Train Epoch: 1	Batch: [ 3360/ 4954 ( 68%)]	Loss: 0.055351
Training accuracy: 1.0
Train Epoch: 1	Batch: [ 3840/ 4954 ( 77%)]	Loss: 0.521227
Training accuracy: 0.8125
Train Epoch: 1	Batch: [ 4320/ 4954 ( 87%)]	Loss: 0.350508
Training accuracy: 0.875
Train Epoch: 1	Batch: [ 4800/ 4954 ( 97%)]	Loss: 0.281600
Training accuracy: 0.9375


/home/dua/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if __name__ == '__main__':



Test: Average loss: 0.2705, Accuracy: 493/546 (90.29%)

Train Epoch: 2	Batch: [    0/ 4954 (  0%)]	Loss: 0.938285
Training accuracy: 0.625
Train Epoch: 2	Batch: [  480/ 4954 ( 10%)]	Loss: 0.034274
Training accuracy: 1.0
Train Epoch: 2	Batch: [  960/ 4954 ( 19%)]	Loss: 0.103373
Training accuracy: 1.0
Train Epoch: 2	Batch: [ 1440/ 4954 ( 29%)]	Loss: 0.170745
Training accuracy: 0.9375
Train Epoch: 2	Batch: [ 1920/ 4954 ( 39%)]	Loss: 0.366147
Training accuracy: 0.875
Train Epoch: 2	Batch: [ 2400/ 4954 ( 48%)]	Loss: 0.381887
Training accuracy: 0.8125
Train Epoch: 2	Batch: [ 2880/ 4954 ( 58%)]	Loss: 0.275359
Training accuracy: 0.9375
Train Epoch: 2	Batch: [ 3360/ 4954 ( 68%)]	Loss: 0.323073
Training accuracy: 0.875
Train Epoch: 2	Batch: [ 3840/ 4954 ( 77%)]	Loss: 0.145242
Training accuracy: 0.9375
Train Epoch: 2	Batch: [ 4320/ 4954 ( 87%)]	Loss: 0.131137
Training accuracy: 1.0
Train Epoch: 2	Batch: [ 4800/ 4954 ( 97%)]	Loss: 0.239530
Training accuracy: 0.875

Test: Average loss: 0.2016, Acc

Train Epoch: 10	Batch: [ 2880/ 4954 ( 58%)]	Loss: 0.268618
Training accuracy: 0.875
Train Epoch: 10	Batch: [ 3360/ 4954 ( 68%)]	Loss: 0.013359
Training accuracy: 1.0
Train Epoch: 10	Batch: [ 3840/ 4954 ( 77%)]	Loss: 0.008689
Training accuracy: 1.0
Train Epoch: 10	Batch: [ 4320/ 4954 ( 87%)]	Loss: 0.028512
Training accuracy: 1.0
Train Epoch: 10	Batch: [ 4800/ 4954 ( 97%)]	Loss: 0.003516
Training accuracy: 1.0

Test: Average loss: 0.2025, Accuracy: 520/546 (95.24%)

Train Epoch: 11	Batch: [    0/ 4954 (  0%)]	Loss: 0.006079
Training accuracy: 1.0
Train Epoch: 11	Batch: [  480/ 4954 ( 10%)]	Loss: 0.000497
Training accuracy: 1.0
Train Epoch: 11	Batch: [  960/ 4954 ( 19%)]	Loss: 0.002158
Training accuracy: 1.0
Train Epoch: 11	Batch: [ 1440/ 4954 ( 29%)]	Loss: 0.319641
Training accuracy: 0.9375
Train Epoch: 11	Batch: [ 1920/ 4954 ( 39%)]	Loss: 0.029221
Training accuracy: 1.0
Train Epoch: 11	Batch: [ 2400/ 4954 ( 48%)]	Loss: 0.008100
Training accuracy: 1.0
Train Epoch: 11	Batch: [ 2880/ 4954 

Train Epoch: 19	Batch: [    0/ 4954 (  0%)]	Loss: 0.001066
Training accuracy: 1.0
Train Epoch: 19	Batch: [  480/ 4954 ( 10%)]	Loss: 0.007915
Training accuracy: 1.0
Train Epoch: 19	Batch: [  960/ 4954 ( 19%)]	Loss: 0.004114
Training accuracy: 1.0
Train Epoch: 19	Batch: [ 1440/ 4954 ( 29%)]	Loss: 0.024853
Training accuracy: 1.0
Train Epoch: 19	Batch: [ 1920/ 4954 ( 39%)]	Loss: 0.002382
Training accuracy: 1.0
Train Epoch: 19	Batch: [ 2400/ 4954 ( 48%)]	Loss: 0.001601
Training accuracy: 1.0
Train Epoch: 19	Batch: [ 2880/ 4954 ( 58%)]	Loss: 0.029699
Training accuracy: 1.0
Train Epoch: 19	Batch: [ 3360/ 4954 ( 68%)]	Loss: 0.000673
Training accuracy: 1.0
Train Epoch: 19	Batch: [ 3840/ 4954 ( 77%)]	Loss: 0.002248
Training accuracy: 1.0
Train Epoch: 19	Batch: [ 4320/ 4954 ( 87%)]	Loss: 0.000057
Training accuracy: 1.0
Train Epoch: 19	Batch: [ 4800/ 4954 ( 97%)]	Loss: 0.002484
Training accuracy: 1.0

Test: Average loss: 0.2489, Accuracy: 527/546 (96.52%)

Train Epoch: 20	Batch: [    0/ 4954 (  0%

Train Epoch: 27	Batch: [ 3360/ 4954 ( 68%)]	Loss: 0.000079
Training accuracy: 1.0
Train Epoch: 27	Batch: [ 3840/ 4954 ( 77%)]	Loss: 0.001297
Training accuracy: 1.0
Train Epoch: 27	Batch: [ 4320/ 4954 ( 87%)]	Loss: 0.000220
Training accuracy: 1.0
Train Epoch: 27	Batch: [ 4800/ 4954 ( 97%)]	Loss: 0.000168
Training accuracy: 1.0

Test: Average loss: 0.3184, Accuracy: 527/546 (96.52%)

Train Epoch: 28	Batch: [    0/ 4954 (  0%)]	Loss: 0.000012
Training accuracy: 1.0
Train Epoch: 28	Batch: [  480/ 4954 ( 10%)]	Loss: 0.000025
Training accuracy: 1.0
Train Epoch: 28	Batch: [  960/ 4954 ( 19%)]	Loss: 0.003796
Training accuracy: 1.0
Train Epoch: 28	Batch: [ 1440/ 4954 ( 29%)]	Loss: 0.000015
Training accuracy: 1.0
Train Epoch: 28	Batch: [ 1920/ 4954 ( 39%)]	Loss: 0.000007
Training accuracy: 1.0
Train Epoch: 28	Batch: [ 2400/ 4954 ( 48%)]	Loss: 0.000382
Training accuracy: 1.0
Train Epoch: 28	Batch: [ 2880/ 4954 ( 58%)]	Loss: 0.000168
Training accuracy: 1.0
Train Epoch: 28	Batch: [ 3360/ 4954 ( 68%

Train Epoch: 36	Batch: [  960/ 4954 ( 19%)]	Loss: 0.000012
Training accuracy: 1.0
Train Epoch: 36	Batch: [ 1440/ 4954 ( 29%)]	Loss: 0.000949
Training accuracy: 1.0
Train Epoch: 36	Batch: [ 1920/ 4954 ( 39%)]	Loss: 0.000206
Training accuracy: 1.0
Train Epoch: 36	Batch: [ 2400/ 4954 ( 48%)]	Loss: 0.000019
Training accuracy: 1.0
Train Epoch: 36	Batch: [ 2880/ 4954 ( 58%)]	Loss: 0.000047
Training accuracy: 1.0
Train Epoch: 36	Batch: [ 3360/ 4954 ( 68%)]	Loss: 0.000011
Training accuracy: 1.0
Train Epoch: 36	Batch: [ 3840/ 4954 ( 77%)]	Loss: 0.000831
Training accuracy: 1.0
Train Epoch: 36	Batch: [ 4320/ 4954 ( 87%)]	Loss: 0.000019
Training accuracy: 1.0
Train Epoch: 36	Batch: [ 4800/ 4954 ( 97%)]	Loss: 0.001691
Training accuracy: 1.0

Test: Average loss: 0.3621, Accuracy: 527/546 (96.52%)

Train Epoch: 37	Batch: [    0/ 4954 (  0%)]	Loss: 0.000247
Training accuracy: 1.0
Train Epoch: 37	Batch: [  480/ 4954 ( 10%)]	Loss: 0.000764
Training accuracy: 1.0
Train Epoch: 37	Batch: [  960/ 4954 ( 19%